<a href="https://colab.research.google.com/github/vanderbilt-ml/50-nelson-mlproj-waittime/blob/main/wait_time_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wait Time Prediction


## Background

Recently when planning an upcoming vacation I discovered that a company called Touringplans (touringplans.com) has many publically available data sets with captured wait times for attractions at Walt Disney World in Florida dating back to 2015. I'm intrigued by this data and am interested in building a predective model using the historical wait time data to help forecast future wait times.

## Project Description

Using the captured historical wait time data I would like to create a predictive model that will help myself to understand future wait times of attractions at Walt Disney World in Florida.

The following columns represent my core data:


*   Date: The captured data date
*   DateTime: The captured data datetime
*   SActMin: The actual wait time at the given datetime (if catpured)
*   SPostMin: The posted wait time at the given datetime



Via the metadata.csv file we have loads of relevant information for each date our data has been collected for. I will be able to utilize this data by joining metadata.csv and our sample data via the DATE column. Within this file are important pieces of information like:

*   DayOfWeek
*   DayOfYear
*   WeekOfYear
*   MonthOfYear
*   Season
*   MaxTemp
*   MinTemp
*   MeanTemp



## Performance Metric
Given the abundance of available data I imagine I will be able to split the data into both training and testing data. I would like to be able to create a predictive model with somewhere in the 80-90% accuracy range. At this point however I have no clue if that is possible.

## Required Imports

In [342]:
#tables and visualizations
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#machine learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler
from sklearn import config_context
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score

## Load Data

The metadata is stored in a separate file; loading in both predictive data and metadata, then combining.

In [343]:
wait_time_raw_data = pd.read_csv('https://raw.githubusercontent.com/vanderbilt-ml/50-nelson-mlproj-waittime/main/big_thunder_mtn.csv')
metadata = pd.read_csv('https://raw.githubusercontent.com/vanderbilt-ml/50-nelson-mlproj-waittime/main/provided_data/metadata.csv')
# To minimize training time for now I've limited the number of metadata columns I'm using to just the following:
metadata = metadata[['DATE', 'DAYOFWEEK', 'DAYOFYEAR', 'WEEKOFYEAR','MONTHOFYEAR']] #, 'SEASON']]
metadata.rename(columns = {'DATE':'date'},  inplace=True)
wait_time_data = pd.merge(wait_time_raw_data, metadata, on ='date')
# Currently having some issues with datetime objects during training, here's some of my attempts to remedy the issue
# wait_time_data['date'] = pd.to_datetime(wait_time_data['date'])
# wait_time_data['datetime'] = pd.to_datetime(wait_time_data['datetime'])
# wait_time_data['datetime'] = np.Timestamp(np.datetime64(wait_time_data['datetime']))
# wait_time_data['datetime'] = wait_time_data['datetime'].values.astype('datetime64[D]')
# wait_time_data['date'] = wait_time_data['date'].values.astype('datetime64[D]')
print(wait_time_data.shape)
print(wait_time_data.head())

(268969, 8)
         date             datetime  SACTMIN  SPOSTMIN  DAYOFWEEK  DAYOFYEAR  \
0  01/01/2015  2015-01-01 08:02:13      NaN       5.0          5          0   
1  01/01/2015  2015-01-01 08:09:12      NaN      15.0          5          0   
2  01/01/2015  2015-01-01 08:16:12      NaN      20.0          5          0   
3  01/01/2015  2015-01-01 08:23:12      NaN      20.0          5          0   
4  01/01/2015  2015-01-01 08:23:53      NaN      20.0          5          0   

   WEEKOFYEAR  MONTHOFYEAR  
0           0            1  
1           0            1  
2           0            1  
3           0            1  
4           0            1  


## Data Cleaning and Validation

In [344]:
wait_time_data.isna().sum()


date                0
datetime            0
SACTMIN        260224
SPOSTMIN         8745
DAYOFWEEK           0
DAYOFYEAR           0
WEEKOFYEAR          0
MONTHOFYEAR         0
dtype: int64

In [345]:
wait_time_data.shape

(268969, 8)

We have many entries with -999 entered as their SPOSTMIN entry. I'll go ahead and drop those. 

In [346]:
wait_time_data = wait_time_data[wait_time_data.SPOSTMIN != -999]
print(wait_time_data.shape)

(246931, 8)


The SACTMIN and SPOSTMIN entries are mutually exclusive. Meaning for every data entry only one of the columns will have data. The SACTMIN should be more valuable data than the SPOSTMIN column; I'm not sure yet how I should handle this so I'll leave them as-is for now

Dropping any columns that are completely empty

In [347]:
wait_time_data.dropna(how='all', axis=1, inplace=True)
display(wait_time_data)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,date,datetime,SACTMIN,SPOSTMIN,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR
0,01/01/2015,2015-01-01 08:02:13,NaN,5.0,5,0,0,1
1,01/01/2015,2015-01-01 08:09:12,NaN,15.0,5,0,0,1
2,01/01/2015,2015-01-01 08:16:12,NaN,20.0,5,0,0,1
3,01/01/2015,2015-01-01 08:23:12,NaN,20.0,5,0,0,1
4,01/01/2015,2015-01-01 08:23:53,NaN,20.0,5,0,0,1
...,...,...,...,...,...,...,...,...
268962,08/31/2021,2021-08-31 20:32:54,NaN,10.0,3,242,35,8
268963,08/31/2021,2021-08-31 20:40:13,NaN,10.0,3,242,35,8
268964,08/31/2021,2021-08-31 20:47:24,NaN,10.0,3,242,35,8
268965,08/31/2021,2021-08-31 20:54:12,NaN,10.0,3,242,35,8


## Feature Engineering

For now, given the mutually exclusive data relationship between SACTMIN and SPOSTMIN I am going to collapse them into one column. SACTMIN represents human-captured wait time (someone stood in line and captured their wait length) and SPOSTMIN captures the posted wait time. In my opion this makes SACTMIN data more valuable, but given the small percentage of data entries that SACTMIN data makes up I'm not sure what other approach to take at this point.

In [348]:
wait_time_data[wait_time_data["SACTMIN"].notna()].head()

,date,datetime,SACTMIN,SPOSTMIN,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR
63,01/01/2015,2015-01-01 14:55:16,37.0,NaN,5,0,0,1
142,01/02/2015,2015-01-02 08:40:32,3.0,NaN,6,1,0,1
152,01/02/2015,2015-01-02 09:30:53,35.0,NaN,6,1,0,1
160,01/02/2015,2015-01-02 10:16:26,47.0,NaN,6,1,0,1
190,01/02/2015,2015-01-02 13:16:31,54.0,NaN,6,1,0,1


In [349]:
wait_time_data['wait'] = pd.to_numeric(wait_time_data[['SACTMIN', 'SPOSTMIN']].bfill(axis=1).iloc[:, 0])
wait_time_data[wait_time_data["SACTMIN"].notna()].head()

,date,datetime,SACTMIN,SPOSTMIN,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,wait
63,01/01/2015,2015-01-01 14:55:16,37.0,NaN,5,0,0,1,37.0
142,01/02/2015,2015-01-02 08:40:32,3.0,NaN,6,1,0,1,3.0
152,01/02/2015,2015-01-02 09:30:53,35.0,NaN,6,1,0,1,35.0
160,01/02/2015,2015-01-02 10:16:26,47.0,NaN,6,1,0,1,47.0
190,01/02/2015,2015-01-02 13:16:31,54.0,NaN,6,1,0,1,54.0


In [350]:
wait_time_data = wait_time_data.drop('SACTMIN', axis=1)
wait_time_data = wait_time_data.drop('SPOSTMIN', axis=1)
wait_time_data.head()

,date,datetime,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,wait
0,01/01/2015,2015-01-01 08:02:13,5,0,0,1,5.0
1,01/01/2015,2015-01-01 08:09:12,5,0,0,1,15.0
2,01/01/2015,2015-01-01 08:16:12,5,0,0,1,20.0
3,01/01/2015,2015-01-01 08:23:12,5,0,0,1,20.0
4,01/01/2015,2015-01-01 08:23:53,5,0,0,1,20.0


## Datetime Issues (don't run)

In [327]:
print(wait_time_data.dtypes)

date            object
datetime        object
DAYOFWEEK        int64
DAYOFYEAR        int64
WEEKOFYEAR       int64
MONTHOFYEAR      int64
wait           float64
dtype: object


In [317]:
# wait_time_data['date'] =  pd.to_datetime(wait_time_data['date'], format='%m/%d/%Y')
wait_time_data['datetime'] =  pd.to_datetime(wait_time_data['datetime'], format='%Y-%m-%d %H:%M:%S')

In [293]:
print(wait_time_data.dtypes)

date           datetime64[ns]
datetime       datetime64[ns]
DAYOFWEEK               int64
DAYOFYEAR               int64
WEEKOFYEAR              int64
MONTHOFYEAR             int64
wait                  float64
dtype: object


In [116]:
# wait_time_data['date'] =  wait_time_data['date'].values.astype('datetime64[D]').dtype
# wait_time_data['datetime'] =  wait_time_data['datetime'].values.astype('datetime64[D]').dtype

In [210]:
# print(wait_time_data.dtypes)
wait_time_data = wait_time_data.drop('date', axis=1)
wait_time_data = wait_time_data.drop('datetime', axis=1)
# wait_time_data.head()

In [247]:
wait_time_data['wait'] = wait_time_data['wait'].astype(np.int64)

In [197]:
# print(wait_time_data['date'].map(type) == pd.datetime)

NameError: ignored

In [274]:
wait_time_data['date'] = wait_time_data['date'].astype(float)
wait_time_data['datetime'] =  wait_time_data['datetime'].astype(float)

TypeError: ignored

In [ ]:
wait_time_data['date']

In [294]:
print(wait_time_data.dtypes)

date           datetime64[ns]
datetime       datetime64[ns]
DAYOFWEEK               int64
DAYOFYEAR               int64
WEEKOFYEAR              int64
MONTHOFYEAR             int64
wait                  float64
dtype: object


In [295]:
wait_time_data

,date,datetime,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,wait
0,2015-01-01,2015-01-01 08:02:13,5,0,0,1,5.0
1,2015-01-01,2015-01-01 08:09:12,5,0,0,1,15.0
2,2015-01-01,2015-01-01 08:16:12,5,0,0,1,20.0
3,2015-01-01,2015-01-01 08:23:12,5,0,0,1,20.0
4,2015-01-01,2015-01-01 08:23:53,5,0,0,1,20.0
...,...,...,...,...,...,...,...
268962,2021-08-31,2021-08-31 20:32:54,3,242,35,8,10.0
268963,2021-08-31,2021-08-31 20:40:13,3,242,35,8,10.0
268964,2021-08-31,2021-08-31 20:47:24,3,242,35,8,10.0
268965,2021-08-31,2021-08-31 20:54:12,3,242,35,8,10.0


## Dropping datetimes

Still really struggling to understand the issues I'm running into while including the datetime objects in my data. Removing them for now.

In [351]:
wait_time_data = wait_time_data.drop('date', axis=1)
wait_time_data = wait_time_data.drop('datetime', axis=1)
wait_time_data.dtypes

DAYOFWEEK        int64
DAYOFYEAR        int64
WEEKOFYEAR       int64
MONTHOFYEAR      int64
wait           float64
dtype: object

## Test Train Split

In [352]:
wait_time_data = wait_time_data.dropna(subset=['wait'])
wait_time_data.shape

(246931, 5)

In [353]:
class_column = 'wait'
random_seed = 2435

wait_time_data = wait_time_data[:100]

X_train, X_test, y_train, y_test = train_test_split(wait_time_data.drop(columns=class_column), wait_time_data[class_column],
                                                    test_size=0.25, random_state=random_seed)#, stratify=wait_time_data[class_column])

In [354]:
wait_time_data.shape

(100, 5)

In [355]:
# X Train
print('On X train: ')
print('X train dimensions: ', X_train.shape)
display(X_train.head())
display(X_train.dtypes)

# X test
print('\nOn X test: ')
print('X test dimensions: ', X_test.shape)
display(X_test.head())
display(X_test.dtypes)

On X train: 
X train dimensions:  (75, 4)


,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR
69,5,0,0,1
50,5,0,0,1
55,5,0,0,1
36,5,0,0,1
6,5,0,0,1


DAYOFWEEK      int64
DAYOFYEAR      int64
WEEKOFYEAR     int64
MONTHOFYEAR    int64
dtype: object


On X test: 
X test dimensions:  (25, 4)


,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR
37,5,0,0,1
11,5,0,0,1
39,5,0,0,1
48,5,0,0,1
9,5,0,0,1


DAYOFWEEK      int64
DAYOFYEAR      int64
WEEKOFYEAR     int64
MONTHOFYEAR    int64
dtype: object

In [356]:
# Y Train
print('On y train: ')
print('y train dimensions: ', y_train.shape)
display(y_train.head())
display(y_train.dtypes)

# Y test
print('\nOn y test: ')
print('y test dimensions: ', y_test.shape)
display(y_test.head())
display(y_test.dtypes)

On y train: 
y train dimensions:  (75,)


69    60.0
50    60.0
55    60.0
36    50.0
6     20.0
Name: wait, dtype: float64

dtype('float64')


On y test: 
y test dimensions:  (25,)


37    50.0
11    20.0
39    50.0
48    85.0
9     20.0
Name: wait, dtype: float64

dtype('float64')

In [357]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import  RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# create an object of the LinearRegression Model
model_LR = LinearRegression()

# fit the model with the training data
# Running into issues training the simple linear regression model because of datetime objects
model_LR.fit(X_train, y_train)

# predict the target on train and test data 
predict_train = model_LR.predict(X_train)
predict_test  = model_LR.predict(y_train)

# Root Mean Squared Error on train and test date
print('RMSE on train data: ', mean_squared_error(y_train, predict_train)**(0.5))
print('RMSE on test data: ',  mean_squared_error(y_test, predict_test)**(0.5))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"


ValueError: ignored

In [358]:
#individual pipelines for differing datatypes
# cat_pipeline = Pipeline(steps=[('cat_impute', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
#                                ('onehot_cat', OneHotEncoder(handle_unknown='ignore'))])

cat_pipeline = Pipeline(steps=[('cat_impute', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                               ('onehot_cat', OneHotEncoder(drop='if_binary'))])
num_pipeline = Pipeline(steps=[('impute_num', SimpleImputer(missing_values=np.nan, strategy='mean')),
                               ('scale_num', StandardScaler())])

In [359]:
#establish preprocessing pipeline by columns
preproc = ColumnTransformer([('cat_pipe', cat_pipeline, make_column_selector(dtype_include=object)),
                             ('num_pipe', num_pipeline, make_column_selector(dtype_include=np.number))],
                             remainder='passthrough')

In [360]:
#generate the whole modeling pipeline with preprocessing
pipe = Pipeline(steps=[('preproc', preproc),
                       ('mdl', LogisticRegression(penalty='elasticnet', solver='saga', tol=0.01))])

#visualization for steps
with config_context(display='diagram'):
    display(pipe)

Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_pipe',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot_cat',
                                                                   OneHotEncoder(drop='if_binary'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f926e7dfd90>),
                                                 ('num_pipe',
                                                  Pipeline(steps=[('impute_num',
                                                                   SimpleImputer()),
                                                                  ('scale_num',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f926e7df4d0>)])),
                ('mdl',
                 LogisticRegression(penalty='elasticnet', solver='saga',
                                    tol=0.01))])

## Cross-validation with hyperparameter tuning

In [361]:
tuning_grid = {'mdl__l1_ratio' : np.linspace(0,1,5),
               'mdl__C': np.logspace(-1, 6, 3) }
grid_search = GridSearchCV(pipe, param_grid = tuning_grid, cv = 5, return_train_score=True)

In [362]:
tuning_grid

{'mdl__C': array([1.00000000e-01, 3.16227766e+02, 1.00000000e+06]),
 'mdl__l1_ratio': array([0.  , 0.25, 0.5 , 0.75, 1.  ])}

In [363]:
# Having issues with fitting due to datetime objects
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat_pipe',
                                                                         Pipeline(steps=[('cat_impute',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot_cat',
                                                                                          OneHotEncoder(drop='if_binary'))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f926e7dfd90>),
                                                                        ('num_pipe',
                                                    

In [364]:
print(grid_search.best_score_)
grid_search.best_params_


0.30666666666666664


{'mdl__C': 316.22776601683796, 'mdl__l1_ratio': 0.25}

In [365]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_mdl__C,param_mdl__l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.009782,0.002678,0.002895,0.000258,0.1,0.0,"{'mdl__C': 0.1, 'mdl__l1_ratio': 0.0}",0.266667,0.266667,0.133333,...,0.240000,0.067987,9,0.333333,0.250000,0.166667,0.316667,0.266667,0.266667,0.058689
1,0.009849,0.001138,0.003625,0.000657,0.1,0.25,"{'mdl__C': 0.1, 'mdl__l1_ratio': 0.25}",0.266667,0.266667,0.266667,...,0.293333,0.032660,3,0.333333,0.250000,0.250000,0.316667,0.316667,0.293333,0.035901
2,0.011084,0.001145,0.005082,0.001876,0.1,0.5,"{'mdl__C': 0.1, 'mdl__l1_ratio': 0.5}",0.066667,0.333333,0.133333,...,0.200000,0.094281,14,0.133333,0.316667,0.166667,0.250000,0.150000,0.203333,0.069442
3,0.009355,0.001168,0.003303,0.000758,0.1,0.75,"{'mdl__C': 0.1, 'mdl__l1_ratio': 0.75}",0.200000,0.266667,0.333333,...,0.280000,0.049889,4,0.150000,0.250000,0.316667,0.250000,0.316667,0.256667,0.061101
4,0.009942,0.001913,0.003225,0.000697,0.1,1.0,"{'mdl__C': 0.1, 'mdl__l1_ratio': 1.0}",0.266667,0.266667,0.133333,...,0.266667,0.073030,6,0.250000,0.250000,0.166667,0.316667,0.316667,0.260000,0.055377
5,0.008582,0.000568,0.002695,0.000028,316.227766,0.0,"{'mdl__C': 316.22776601683796, 'mdl__l1_ratio'...",0.266667,0.266667,0.333333,...,0.253333,0.065320,7,0.333333,0.250000,0.316667,0.250000,0.116667,0.253333,0.076303
6,0.008506,0.000217,0.002674,0.000061,316.227766,0.25,"{'mdl__C': 316.22776601683796, 'mdl__l1_ratio'...",0.266667,0.266667,0.333333,...,0.306667,0.032660,1,0.333333,0.250000,0.316667,0.316667,0.316667,0.306667,0.029059
7,0.008397,0.000181,0.002657,0.000092,316.227766,0.5,"{'mdl__C': 316.22776601683796, 'mdl__l1_ratio'...",0.266667,0.333333,0.333333,...,0.306667,0.032660,1,0.250000,0.316667,0.316667,0.250000,0.316667,0.290000,0.032660
8,0.008359,0.000122,0.002763,0.000109,316.227766,0.75,"{'mdl__C': 316.22776601683796, 'mdl__l1_ratio'...",0.266667,0.333333,0.333333,...,0.280000,0.077746,4,0.250000,0.316667,0.316667,0.116667,0.316667,0.263333,0.077746
9,0.008442,0.000463,0.002656,0.000056,316.227766,1.0,"{'mdl__C': 316.22776601683796, 'mdl__l1_ratio'...",0.266667,0.133333,0.133333,...,0.213333,0.077746,12,0.250000,0.166667,0.166667,0.316667,0.150000,0.210000,0.063770
